In [1]:
import gc
import pandas as pd
import numpy as np
import pyarrow as pa
import tensorflow as tf
from pyarrow import parquet as pq
from collections import defaultdict

In [2]:
train = pd.read_csv('../Data/Train/sales_train_validation.csv')
train = train.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis =1)
train.index = train.id
train = train.drop('id',axis = 1)

In [16]:
def _rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def _get_time_tensor(arr,window_size):
    tmp = _rolling_window(arr,window_size+1)
    Xtensor = tmp[:,:-1]
    Ytensor = tmp[:,-1]
    return (Xtensor.reshape(-1,window_size,1),Ytensor)

def np_to_time_tensor_generator(arr,windowSize):
    if np.ndim(arr) > 1:
        for ix,v in enumerate(arr):
            yield _get_time_tensor(v,windowSize)
    else:
        yield _get_time_tensor(arr,windowSize) 

def _get_item_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[:3])

def _get_store_id(fullIndex):
    tmp = fullIndex.split('_')
    return '_'.join(tmp[3:5])

def _get_cate_info(fullIndex,cateInfoDir):
    item_id = _get_item_id(fullIndex)
    store_id = _get_store_id(fullIndex)
    return pd.read_parquet(cateInfoDir,filters = [("item_id",'=',str(item_id)),("store_id",'=',str(store_id))])

def _get_events(calendar_dir,sdate,edate):
    df = pd.read_csv(calendar_dir)
    df['d_num'] = df.d.apply(lambda x: x.replace('d_','')).astype('int')
    return df[df.d_num.apply(lambda x: x <= edate and x >= sdate)]

def _get_future_events(calendar_dir, sdate,duration):
    edate = sdate+duration-1
    return _get_events(calendar_dir,sdate,edate)


In [17]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000
gen = np_to_time_tensor_generator(train.head(2).values,10)
# train_univariate = tf.data.Dataset.from_tensor_slices(train.head(1).values[0])
# train_univariate = train_univariate.map(lambda x: tf.py_function(_get_time_tensor,inp = [x,10],Tout= (tf.float32,tf.float32)))
train_univariate = tf.data.Dataset.from_generator(
    np_to_time_tensor_generator,
    (tf.float32,tf.float32),
    output_shapes = (tf.TensorShape(None),tf.TensorShape(None)),
    args = (train.head(2).values,10))
train_univariate = train_univariate.batch(BATCH_SIZE).repeat()

In [14]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=(10,1)),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

In [15]:

EVALUATION_INTERVAL = 5
EPOCHS = 10

simple_lstm_model.fit(
    train_univariate,
    epochs=EPOCHS,
    steps_per_epoch= EVALUATION_INTERVAL,
    validation_data=train_univariate, 
    validation_steps=50
    )

Train for 5 steps, validate for 50 steps
Epoch 1/10
1/5 [=====>........................] - ETA: 0s

ValueError: as_list() is not defined on an unknown TensorShape.